In [26]:
import urllib
import time
from urllib.request import urlopen

def getLinks(html, max_links=10):
    url = []
    cursor = 0
    nlinks=0
    while (cursor>=0 and nlinks<max_links):
        start_link = html.find("a href=",cursor)
        if start_link==-1:
            return url
        start_quote = html.find('"', start_link)
        end_quote = html.find('"', start_quote + 1)
        url.append(html[start_quote + 1: end_quote])
        cursor = end_quote+1
        nlinks = nlinks +1
    return url

class Spider:
    def __init__(self,starting_url,crawl_domain,max_iter):
        self.crawl_domain = crawl_domain
        self.max_iter = max_iter
        self.links_to_crawl=[]
        self.links_to_crawl.append(starting_url)
        self.links_visited=[]
        self.collection=[]
        
    def retrieveHtml(self):
        try:
            socket = urllib.request.urlopen(self.url)
            encoding = socket.headers.get_content_charset()
            if encoding is None:
                    encoding = "utf-8"
            self.html = socket.read().decode(encoding)
            return 0
        except UnicodeDecodeError:
            print ("Bad Encoding")
            return -1
        except urllib.error.HTTPError:
            # Most probably an url not found 404, possibly due to malformating of the links in retrieveAndValidateLinks
            print ("Broken Link")
            return -1
    def storeHtml(self):
        doc = {}
        doc['url'] = self.url
        doc['date'] = time.strftime("%d/%m/%Y")
        doc['html'] = self.html
        self.collection.append(doc)          
   
    def retrieveAndValidateLinks(self):
        tmpList=[]
        items = getLinks(self.html)
        # Check the validity of a link
        for item in items:
            item = item.strip('"')
            if self.crawl_domain in item:
                tmpList.append(item)
            if not(":") in item: #Take care of http:// https:// and mailto:
                tmpList.append(self.crawl_domain+item)
        # Check that the link has not been previously retrieved or is currently on the links_to_crawl list
        for item in tmpList:
            if item not in self.links_visited:
                if item not in self.links_to_crawl:
                    self.links_to_crawl.append(item)
                    print ('Adding: '+item)
                
     
        
    def run(self):
        while (len(self.links_to_crawl)>0 and len(self.collection)<self.max_iter):
            
            self.url = self.links_to_crawl.pop(0)
            print ("Visiting: "+ self.url)
            self.links_visited.append(self.url)
            if self.retrieveHtml()>=0:
                self.storeHtml()
                self.retrieveAndValidateLinks()

In [27]:
spider = Spider('https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/','https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/',20)
spider.run()

Visiting: https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/
Adding: https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/no-login-user-bar
Adding: https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal//
Adding: https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal//venta-viviendas/barcelona-provincia/mapa
Adding: https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal//venta-viviendas/barcelona-barcelona/mapa
Adding: https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal//venta-viviendas/barcelona/sants-montjuic/mapa
Adding: https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal//venta-viviendas/barcelona/sants-montjuic/el-poble-sec-parc-de-montjuic/
Adding: https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal//venta-viviendas/barcelona/sants-montjuic/hostafrancs/
Adding: https://www.id

In [16]:
spider.collection[1]['url']

'https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal//'

In [21]:
# <a class="" href="/venta-viviendas/barcelona/sants-montjuic/sants-badal/pagina-3.htm">3</a>

def getLinks_pages(html,main_url, max_links=10):
    url = []
    cursor = 0
    nlinks=0
    while (cursor>=0 and nlinks<max_links):
        start_link = html.find("a href="+str(main_url),cursor)
        if start_link==-1:
            return url
        start_quote = html.find('"', start_link)
        end_quote = html.find('"', start_quote + 1)
        url.append(html[start_quote + 1: end_quote])
        cursor = end_quote+1
        nlinks = nlinks +1
    return url

In [31]:
source=urllib.request.urlopen('https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/')

In [69]:
import re
url='https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/'
# html
try:
    socket = urllib.request.urlopen(url)
    encoding = socket.headers.get_content_charset()
    if encoding is None:
            encoding = "utf-8"
    html2 = socket.read().decode(encoding)
#     return 0
except UnicodeDecodeError:
    print ("Bad Encoding")
#     return -1
except urllib.error.HTTPError:
    # Most probably an url not found 404, possibly due to malformating of the links in retrieveAndValidateLinks
    print ("Broken Link")
#     return -1


In [70]:

list(set(re.findall(r'pagina-\d*',html2)))
print(html2)

<!DOCTYPE html>
<!--[if IE 8]><html class="ie8" lang="es" env="es" username=""><![endif]--> <!--[if IE 9]><html class="ie9" lang="es" env="es" username=""><![endif]--> <!--[if gt IE 9]><!--> <html lang="es" env="es" username="" data-userAuth="false"><!--<![endif]--> <head> <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"> <title>Casas y pisos en Sants - Badal, Barcelona &#8212; idealista</title> <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"> <meta name="description" content="133 casas y pisos en Sants - Badal, Barcelona, a partir de 120.000 euros de particulares e inmobiliarias. casas y pisos en Sants - Badal, Barcelona: anuncios de particular a particular y de agencias inmobiliarias."> <meta name="keywords" content="Viviendas Barcelona, viviendas de particulares Barcelona, viviendas de inmobiliarias Barcelona, fotos de viviendas Barcelona"> <meta name="author" content="idealista.com"> <meta http-equiv="cleartype" content="on"> <meta name="pragma" cont

In [64]:
from lxml import html
import requests
page = requests.get('https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/')
tree = html.fromstring(page.content)


In [73]:
tree.xpath('@class=items-container')

False

# Beautiful soup


In [85]:
import requests

page = requests.get("https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/")
page.status_code #200 is good

200

In [169]:
from bs4 import BeautifulSoup
import requests

page = requests.get("https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/")
page.status_code #200 is good

soup = BeautifulSoup(page.content, 'html.parser')
container=soup.find_all('div', class_='item-info-container')
pags=soup.find_all('div',class_="pagination")
item=container[0]



In [170]:
list(set(re.findall(r'pagina-\d*.htm',str(pags))))
a=get_pages_barri(soup)
print(a)

['pagina-2.htm', 'pagina-5.htm', 'pagina-3.htm', 'pagina-4.htm']


In [109]:
print(type(str(item)))
re.findall(r'<span class="item-price">\w*.?\w*',str(item))
print(len('<span class="item-price">'))

<class 'str'>
25


In [47]:
from bs4 import BeautifulSoup
import requests
import re

def get_pages_barri(soup):
    pags=soup.find_all('div',class_="pagination")
    extension=list(set(re.findall(r'pagina-\d*.htm',str(pags))))
    return extension

def get_info_container(soup):
    container=soup.find_all('div', class_='item-info-container')
    prices=[]
    sizes=[]
    Ids=[]

    for item in container:

        item=str(item)
        price=re.findall(r'<span class="item-price">\d+.?\d+.?\d+',item)[0][25:]
        price=str.replace(price,'.','')
        size=re.findall(r'<span class="item-detail">\d+.?\d+ <small>m²',item)[0][26:-10]
        Id=re.findall(r'href="/inmueble/\d*',item)[0][16:]
        Ids.append(Id)
        sizes.append(size)
        prices.append(price)
    return Ids,prices,sizes

def get_next_page(soup):
    pags=soup.find_all(class_="icon-arrow-right-after")
    if len(pags) != 0:
        return re.findall(r'pagina-\d*.htm',str(pags))[0]
    else:
        return 'stop'

In [31]:
page = requests.get("https://www.idealista.com/venta-viviendas/barcelona/sants-montjuic/sants-badal/")
print ('good' if page.status_code==200 else 'bad') #200 is good
soup = BeautifulSoup(page.content, 'html.parser')


good


In [36]:
pags=soup.find_all(class_="icon-arrow-right-after")
len(pags)
re.findall(r'pagina-\d*.htm',str(pags))[0]


'pagina-2.htm'

In [49]:
import pandas as pd
import numpy as np
file=pd.read_csv('url-neighborhood.csv')
tipus='venta'
# tipus='alquiler'
for distric,nei,path in zip(file['District'],file['Neighborhood'],file['Idealista-path']):

        
    page = requests.get("https://www.idealista.com/"+tipus+"-viviendas/barcelona/"+str(path))
    soup = BeautifulSoup(page.content, 'html.parser')

    next_pages=get_pages_barri(soup)
    ID,Price,Size=get_info_container(soup)
    ID=np.array(ID)
    Price=np.array(Price)
    Size=np.array(Size)
    print(1)
    next_page=get_next_page(soup)
    while next_page !='stop':
        print(next_page)
#     for next_page in next_pages:

        page = requests.get("https://www.idealista.com/"+tipus+"-viviendas/barcelona/"+str(path)+str(next_page))
        soup = BeautifulSoup(page.content, 'html.parser')
        next_pages=get_pages_barri(soup)
        ID_,Price_,Size_=get_info_container(soup)

        ID=np.r_[ID,ID_]
        Price=np.r_[Price,Price_]
        Size=np.r_[Size,Size_]
        next_page=get_next_page(soup)
        
    data=np.c_[ID.T,Price.T,Size.T]
    data=pd.DataFrame(data,columns=['ID','PRICE(€)','SIZE(m^2)'])
    pd.DataFrame.to_csv(data,'./output-'+tipus+'/zone'+str(distric)+'-'+str(nei)+'.csv',index=False)


1
pagina-2.htm
pagina-3.htm
pagina-4.htm
pagina-5.htm
1
pagina-2.htm
pagina-3.htm
pagina-4.htm
1
pagina-2.htm
pagina-3.htm
pagina-4.htm
pagina-5.htm
pagina-6.htm
pagina-7.htm


In [ ]:
path

In [2]:
url_alquiler='https://www.idealista.com/alquiler-viviendas/barcelona/sants-montjuic/sants-badal/'
page=requests.get(url_alquiler)

In [50]:
print ('good' if page.status_code==200 else 'bad') #200 is good
soup = BeautifulSoup(page.content, 'html.parser')
print(get_pages_barri(soup))
print(get_info_container(soup))

good
['pagina-3.htm', 'pagina-2.htm']
(['38778457', '38476642', '38777759', '38559458', '38776418', '38775095', '38773320', '38771885', '38469533', '38712990', '32897413', '37931423', '38153797', '33085324', '38108980', '38744112', '38534537', '38617675', '38556456', '38151087', '38153449', '38677705', '38676813', '25483123', '38171320', '35283802', '38664723', '38267822', '38151897', '35283855'], ['1100', '1600', '765', '900', '1250', '1300', '1000', '1000', '875', '1000', '1375', '900', '1100', '825', '1300', '1100', '900', '790', '875', '1100', '1100', '1200', '1800', '1100', '1000', '1200', '1350', '1440', '1000', '1200'], ['66', '97', '45', '45', '71', '75', '70', '70', '85', '67', '100', '65', '65', '72', '50', '82', '70', '68', '70', '90', '65', '120', '98', '80', '90', '60', '119', '95', '95', '50'])


In [56]:
ide,preu,tamany=get_info_container(soup)

In [72]:
import numpy as np
import pandas as pd
ide=np.array(ide,dtype=int)
preu=np.array(preu,dtype=int)
tamany=np.array(tamany,dtype=int)
data=np.c_[ide.T,preu.T,tamany.T]
data=pd.DataFrame(data,columns=['ID','PRICE(€)','SIZE(m^2)'])
pd.DataFrame.to_csv(data,'prova.csv',index=False)